In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re
import json

with open("/home/jovyan/work/.credentials.json", "r") as credential:
    credential = json.load(credential)
    url = credential["url"]

def get_interest(column):
    """return string that matches the pattern"""
    concept_names = set()
    pattern = r'^(a|b|c|d|e|A|B|C|D|E).*(heart).*'
    for row in column:
        if re.match(pattern, row):
            concept_names.add(row)
    return concept_names

In [2]:
# DB 연결
engine = create_engine(url, connect_args={'options': '-csearch_path={}'.format('de')})
df_condition = pd.read_sql_table(
    "condition_occurrence",
    con=engine
)
df_concept = pd.read_sql_table(
    "concept",
    con=engine
)

In [3]:
# 각 테이블에서 필요한 행만 가져오기
df_condition_concept_id = df_condition[["condition_concept_id"]]
df_concept_id_name = df_concept[["concept_id", "concept_name"]]

In [4]:
# Join
df_merged = pd.merge(df_condition_concept_id, 
                     df_concept_id_name, 
                     how="left", 
                     left_on="condition_concept_id", 
                     right_on="concept_id")

In [5]:
# 첫 글자는 (a,b,c,d,e) 문자로 시작하고 중간에 "heart"가 포함된 상병 이름
print(get_interest(df_merged["concept_name"]))

{'Chronic congestive heart failure'}
